# My Second Agent: Parse LPs (v0.2.0)

**Dr. Dave Wanik - Operations and Information Management - University of Connecticut**

So our first agent was to just write a linear programming problem - this agent works to parse a word problem into its main elements: objective function, decision variables, constraints. Then it stacks them in for loops and solves all possible combinations and shows the output.

## 🛠️ Add an `lp_parser` sub-package to *agent-toolkit*

### 1 ️⃣  Create the folder structure

```
agent-toolkit/               ← repo root (already exists)
└── src/
    ├── agent_toolkit/       ← existing code
    └── lp_parser/           ← NEW
        ├── __init__.py
        └── core.py
```

> **How:**  
> *In VS Code Explorer → Right-click `src/` → “New Folder” → name it `lp_parser`, then add both files.*

---

### 2 ️⃣  Fill the files

**`src/lp_parser/__init__.py`**

```python
from .core import parse_word_problem
```

---

**`src/lp_parser/core.py`**

```python
"""
Natural-language ➜ structured JSON extractor for small LP word problems.
Scope (v0): max-profit LPs with ‘≤’ constraints and integer, non-negative variables.
"""

from __future__ import annotations
import os, json
from openai import OpenAI

_JSON_SCHEMA = """
{
  "objective": {"sense": "max", "coeff": {"<var>": <float>, ...}},
  "vars": {"<var>": {"ub": <int>}, ...},
  "constraints": [
    {"name": "<string>", "coeff": {"<var>": <float>, ...}, "rhs": <float>},
    ...
  ]
}
"""

def _llm_extract(problem_text: str, model: str = "gpt-4o") -> str:
    """Call OpenAI with JSON mode and return raw JSON string."""
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": (
                    "Extract the linear-program parameters ONLY as valid JSON "
                    f"matching this schema (no extra keys): {_JSON_SCHEMA}"
                ),
            },
            {"role": "user", "content": problem_text},
        ],
        temperature=0.0,
        response_format={"type": "json_object"},
    )
    return response.choices[0].message.content

def parse_word_problem(problem_text: str, model: str = "gpt-4o") -> dict:
    """Return a Python dict describing the LP or raise ValueError on failure."""
    raw = _llm_extract(problem_text, model=model)
    try:
        lp = json.loads(raw)
    except json.JSONDecodeError as e:
        raise ValueError(f"LLM did not return valid JSON: {e}") from None
    # ⬇️ very light validation (students can improve)
    if "objective" not in lp or "vars" not in lp or "constraints" not in lp:
        raise ValueError("Parsed JSON missing required keys.")
    return lp
```

---

### 3 ️⃣  Wire this new parser into the main package

**Edit** `src/agent_toolkit/__init__.py` (existing):

```python
from .core import run_chat_agent
from brute_force_lp.core import brute_force_lp, sample_lp
from lp_parser.core import parse_word_problem          # ← NEW
```

---

### 4 ️⃣  Add an example notebook (optional)

1. Create `examples/parse_and_solve.ipynb`
2. Demonstrate:

```python
from lp_parser import parse_word_problem
from agent_toolkit import brute_force_lp

prompt = """Veerman Furniture Company makes three kinds of office furniture ..."""
lp = parse_word_problem(prompt)
best, con_df = brute_force_lp(lp)
print(best)
con_df
```

---

### 5 ️⃣  Update `pyproject.toml` dependencies

Add **`openai`** (already there) plus **`jsonschema`** later if you want strict validation.

```toml
dependencies = [
    "openai>=1.0.0",
    "pandas",
    "numpy"
]
```

*(No version bump needed if you’re still at 0.2.0—but feel free.)*

---

### 6 ️⃣  Commit & push

```bash
git add -A
git commit -m "v0.2.0 – add lp_parser sub-package (word-problem ➜ JSON)"
git push origin main
```

---

### 7 ️⃣  Test in Colab

Use this to test your installation and confirm that `parse_word_problem()` returns structured JSON from a word problem.

```python
!pip install --force-reinstall -q git+https://github.com/<your-username>/agent-toolkit.git

from agent_toolkit import parse_word_problem

# Example word problem (Veerman Furniture)
problem = """
Veerman Furniture Company makes three kinds of office furniture: chairs, desks, and tables.
Each product requires labor in fabrication, assembly, and shipping departments. The available
hours per department are: 1850 (fabrication), 2400 (assembly), and 1500 (shipping).
Demand limits are: 360 chairs, 300 desks, 100 tables.
Profit per product is: $15 (chair), $24 (desk), and $18 (table).
Fabrication uses 4/6/2 hours per chair/desk/table,
Assembly uses 3/5/7 hours,
Shipping uses 3/2/4 hours.
"""

# Run the parser
parsed_lp = parse_word_problem(problem)
parsed_lp
```

If working correctly, this should output a dictionary with:
- `"objective"` (sense and profit coefficients)
- `"vars"` (decision variable bounds)
- `"constraints"` (each constraint’s coefficients and RHS)

Once the solver is added later, we’ll plug this JSON into `brute_force_lp()` or `sample_lp()`.


# Try out the agent!

## Old version (v0.1.0)

In [ ]:
# !pip install --force-reinstall --no-cache-dir git+https://github.com/drdww/agent-toolkit.git@v0.1.0

  Cloning https://github.com/drdww/agent-toolkit.git (to revision v0.1.0) to /tmp/pip-req-build-db83b11r
  Running command git clone --filter=blob:none --quiet https://github.com/drdww/agent-toolkit.git /tmp/pip-req-build-db83b11r
  Running command git checkout -q 96b2069a591b5f153dcb8010eb6da14036c63ff3
  Resolved https://github.com/drdww/agent-toolkit.git to commit 96b2069a591b5f153dcb8010eb6da14036c63ff3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 185.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.0/755.0 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 154.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/7

## The new version (v0.2.0)

In [ ]:
# # Later, upgrade to the parser version
# # It is installed like a package! You didn't clone!
# !pip install --quiet --upgrade git+https://github.com/drdww/agent-toolkit.git@v0.2.0


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# ⬇️ 1.  Always start by installing the exact version you need
!pip install --quiet --force-reinstall --no-cache-dir \
    git+https://github.com/drdww/agent-toolkit.git@v0.2.0

# ⬇️ 2.  Securely load your OpenAI key for this Colab session
import os, getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Paste your OpenAI key → ")


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 141.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 207.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 168.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.0/755.0 kB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 118.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 149.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 160.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 185.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 146.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 13

In [ ]:
# ⬇️ 3.  Import the parser from agent-toolkit v0.2.0
from agent_toolkit import parse_word_problem
import json, pprint

problem = """
Veerman Furniture Company makes three kinds of office furniture: chairs, desks, and tables.
Each product requires labor in fabrication (4,6,2), assembly (3,5,7), and shipping (3,2,4).
Available hours: 1850 fabrication, 2400 assembly, 1500 shipping.
Demand limits: 360 chairs, 300 desks, 100 tables.
Profit per unit: $15 chairs, $24 desks, $18 tables.
Goal: maximize total profit for the coming quarter.
"""

lp_json = parse_word_problem(problem)

print("🎯 Parsed LP:")
pprint.pp(lp_json, width=100, compact=True)

🎯 Parsed LP:
{'objective': {'sense': 'max', 'coeff': {'chairs': 15.0, 'desks': 24.0, 'tables': 18.0}},
 'vars': {'chairs': {'ub': 360}, 'desks': {'ub': 300}, 'tables': {'ub': 100}},
 'constraints': [{'name': 'fabrication',
                  'coeff': {'chairs': 4.0, 'desks': 6.0, 'tables': 2.0},
                  'rhs': 1850.0},
                 {'name': 'assembly',
                  'coeff': {'chairs': 3.0, 'desks': 5.0, 'tables': 7.0},
                  'rhs': 2400.0},
                 {'name': 'shipping',
                  'coeff': {'chairs': 3.0, 'desks': 2.0, 'tables': 4.0},
                  'rhs': 1500.0}]}


🎉 Nice work!

You’ve just built **Layer 1**: a reusable `lp_parser` module that converts natural-language word problems into structured JSON that can be used by a solver.

---

## 🧭 Next Steps (coming soon...)

We'll build **Layer 2**: a solver that can take your JSON and return:

- ✅ The optimal values of the decision variables
- ✅ The maximum (or minimum) objective value
- ✅ A summary report of how the solution uses each constraint (LHS vs RHS)

---

## 🧰 Once you have both layers, you’ll be able to:

- 🧠 Use GPT to extract structured LPs from real-world problem descriptions
- 🧮 Solve them using a brute-force or Monte Carlo engine
- 🧾 Generate readable reports (ideal for teaching and debugging)

---

## 💡 Future enhancements (stretch goals)

- 🔄 Swap out the brute-force solver for PuLP or OR-Tools
- 🎯 Add support for equality (`=`) and ≥ constraints
- 📊 Visualize the feasible region or sample distribution
- 🤖 Build an agent-style wrapper (`solve_lp_from_prompt()`) that combines everything

---

Stay tuned for the next notebook — you’re halfway to a full natural language–to–solver pipeline!
